In [4]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import openai
import os
import time
data = pd.read_csv("C:/Users/maira/OneDrive/Desktop/Kaggle Project/train.csv")
openai.api_key = 'API'
questionaire = data
output_file = "C:/Users/maira/OneDrive/Desktop/Kaggle Project/Updated/zeroshot.csv"
#max_calls_per_min = 3
max_tries=5
model = "gpt-4-turbo"
inputs = []
outputs = []

#Initialize
start_time = time.time()
calls_count = 0

for i, row in questionaire.iterrows():
    print(row)
    '''
    #Wait if calls exceed max_calls_per_min
    calls_count += 1
    if calls_count >= max_calls_per_min:
        current_time = time.time() - start_time
        if current_time < 62:
            time.sleep(62 - current_time)
        start_time = time.time()
        calls_count = 0
    '''
    #initialize messages (Prompt 5)
    messages = []
    system_message = """You are an expert in the scientific field. Answer the following multiple-choice question by
                        selecting the most accurate or likely answer. Be sure to consider the relevant context and
                        principles specific to the science discipline associated with the question, such as Physics,
                        Biology, Astronomy, or any other related field. Once you give the answer, go back and verify if it
                        is correct, otherwise correct your answer until you are sure. Answer format: single letter (A, B,
                        C, D, or E)."""
    messages.append({"role": "system", "content": system_message})

    #parse question
    user_message = f"""
    Question: {row['prompt']}
    A: {row['A']}
    B: {row['B']}
    C: {row['C']}
    D: {row['D']}
    E: {row['E']}
    Answer (provide only the letter):
    """
    messages.append({"role": "user", "content": user_message})
    
    #Send message
    for tries in range(max_tries):
        try:
            response = openai.ChatCompletion.create(
                model=model,
                messages=messages,
                temperature = 0)
            break
        except Exception as e:
            print("Error in openai.ChatCompletion.create: ", e," Trying again in 2 minutes. Attempt " + str(tries) + " of " + str(max_tries))
            time.sleep(120)

    #Store outputs
    inputs.append(user_message)
    outputs.append(response["choices"][0]["message"]["content"])

    #Dump outputs
    print("Dumping outputs in", output_file)
    df = pd.DataFrame()
    df["input"] = inputs
    df["output"] = outputs
    df.to_csv(output_file, index = False)

id                                                        0
prompt    Which of the following statements accurately d...
A         MOND is a theory that reduces the observed mis...
B         MOND is a theory that increases the discrepanc...
C         MOND is a theory that explains the missing bar...
D         MOND is a theory that reduces the discrepancy ...
E         MOND is a theory that eliminates the observed ...
answer                                                    D
Name: 0, dtype: object
Dumping outputs in C:/Users/maira/OneDrive/Desktop/Kaggle Project/Updated/zeroshot.csv
id                                                        1
prompt    Which of the following is an accurate definiti...
A         Dynamic scaling refers to the evolution of sel...
B         Dynamic scaling refers to the non-evolution of...
C         Dynamic scaling refers to the evolution of sel...
D         Dynamic scaling refers to the non-evolution of...
E         Dynamic scaling refers to the evolution 

In [5]:
data_predictions = pd.read_csv("C:/Users/maira/OneDrive/Desktop/Kaggle Project/Updated/zeroshot.csv")
data_target = data
data_target.shape
data_predictions.shape
len(data_target) == len(data_predictions)
#assert len(data_target) == len(data_predictions)
diff_count = (data_target['answer'] != data_predictions['output']).sum()
print(f"Number of differing entries: {diff_count}")

Number of differing entries: 29


In [6]:
diff= 200-diff_count
perc= diff/200*100
perc

85.5